In [15]:
# usare wordnet per tradurle e avere i synset 
# https://www.nltk.org/howto/wordnet.html

# https://linguistics.stackexchange.com/questions/19015/italian-is-there-an-authoritative-word-frequency-list

# https://martinweisser.org/corpora_site/word_lists.html

import re
import os.path
import pandas as pd
 

In [35]:
# 1. Extraction of pairs of false-friends words (eng-ita)

path_corpus_ita = f'resource/ita/current_version_morph-it/morph-it_048_utf-8.txt'

path_corpus_eng = f'resource/eng/BNC_lemmafile5.txt'

path_words_ita = f'resource/ita/words_ita.txt'
path_words_eng = f'resource/eng/words_eng.txt'

pos_it = ['ADJ', 'ADV', 'ASP', 'AUX', 'CAU', 'MOD', 'NOUN', 'VER']
words_ita = []
pos_eng = ['j', 'v', 'n','r']
words_eng = []

# 1.1 Extraction of words from corpora ITA - Morph-It! (15k words)

if not os.path.exists(path_words_ita) and os.path.exists(path_corpus_ita):
    with open(path_corpus_ita, 'r', encoding='utf-8') as morph_it_file:
        reader = morph_it_file.readlines()
        for row in reader:
            text, lemma, pos = re.split('\t+', row)
            if lemma not in words_ita:
                pos = pos.replace('\n', '')
                pos = pos.split(":")[0]
                if pos in pos_it:
                    words_ita.append(lemma)

    with open(path_words_ita, 'w', encoding='utf-8') as words_ita_file:
        for word in words_ita:
            words_ita_file.write(word + '\n')

else: 
    with open(path_words_ita, 'r', encoding='utf-8') as words_ita_file:
        reader = words_ita_file.readlines()
        for row in reader:
            words_ita.append(row.replace('\n', ''))


print("Italian words loaded ✔ (", len(words_ita), ")")


# 1.2 Extraction of words from corpora ENG - The British National Corpus (BNC) extracted from WordSmith Tools

# https://lexically.net/wordsmith/support/lemma_lists.html


if not os.path.exists(path_words_eng) and os.path.exists(path_corpus_eng):
    with open(path_corpus_eng, 'r', encoding='utf-8') as corpus_eng_file:
        reader = corpus_eng_file.readlines()
        for row in reader:
            lemma = row.split(" -> ")[0].lower()
            if len(lemma) > 2 and lemma not in words_eng:
                words_eng.append(lemma)

    with open(path_words_eng, 'w', encoding='utf-8') as words_eng_file:
        for word in words_eng:
            words_eng_file.write(word + '\n')

else: 
    with open(path_words_eng, 'r', encoding='utf-8') as words_eng_file:
        reader = words_eng_file.readlines()
        for row in reader:
            words_eng.append(row.replace('\n', ''))

print("English words loaded ✔ (", len(words_eng), ")")



Italian words loaded ✔ ( 16312 )
English words loaded ✔ ( 20280 )


In [ ]:
# 2. Extraction of pairs of false-friends words from words_eng and words_ita




In [27]:
from nltk.corpus import wordnet as wn

print("ITA: ", wn.synsets('casco', lang='ita'))

print("EN: ", wn.synsets('cask'))

# similarità tra lista di synsets?? palmer???

# calcolo della similarità anche con altre metriche

# BONUS: utilizzo fasttext!

ITA:  []
EN:  [Synset('cask.n.01'), Synset('barrel.n.02')]


In [ ]:
# 2. Computation of similarity between pairs of synsets (eng-ita) using lesk algorithm